In [ ]:
! pip install numpy
! pip install openai==1.2.3
! pip install pymongo
! pip install python-dotenv
! pip install prettytable

## Set up

In [162]:
from prettytable import PrettyTable
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "myconfig.env" 
config = dotenv_values(env_name)

# Values for the two prediction functions
user_query = "Snowboards used by Olympic champions"
user_id = int(189)
product_id = int(92) # Shaun White snowboard
num_results = 10


# Connection string
cosmos_conn = config['cosmos_connection_string']
cosmos_client = pymongo.MongoClient(cosmos_conn)

# Database name
database = cosmos_client[config['cosmos_database']]

# Collection names
actual_ratings = database[config['cosmos_actual_ratings']]
predicted_ratings = database[config['cosmos_predicted_ratings']]
product_catalog = database[config['cosmos_product_catalog']]

# OpenAI
openai.api_type = config['openai_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

In [139]:
def generate_embeddings(text):
    
    response = client.embeddings.create(
        input=text, model="embeddings") # need to read from config
    
    embeddings = response.data[0].embedding
        
    return embeddings

In [163]:
def print_vector_search_results(results):
    
    print("---------Vector Search Results: --------")

    # Define the table
    table = PrettyTable()
    table.field_names = ["Product Id", "Name", "Price", "Similarity Score", "Rating"]

    # Add rows to the table
    for product in results:
        table.add_row([
            product['document']['Id'],
            product['document']['Name'],
            product['document']['Price'],
            product['similarityScore'],
            product['document']['rating']
        ])

    # Print the table
    print(table)

In [166]:
def print_predictions_from_product_page(results):
    print("\n--------Current Page Results: ---------")

    # Define the table
    table = PrettyTable()
    table.field_names = ["Product Id", "Name", "Price", "Rating"]

    # Add rows to the table
    for product in results:
        table.add_row([
            product['Id'],
            product['Name'],
            product['Price'],
            product['Rating']
        ])

    # Print the table
    print(table)

## Product Recommendation Functions

In [176]:
def predictions_from_current_product_page(user_id, current_product_id, num_results=4):
    """
    This function displays predicted products for this user excluding the current product.
    """
    
    # Get the predicted products for the user, limit results
    user_predicted_products = predicted_ratings.find_one( 
        { "UserId": user_id },
        {"Predictions": {"$slice": num_results}})

    # Remove the current product from the list
    user_predicted_products = [prediction for prediction in user_predicted_products['Predictions']
        if prediction['ProductId'] != current_product_id]
    
    predicted_products = []

    # Look up recommended products maintaining order of predicted ratings
    for item in user_predicted_products:
        product = product_catalog.find_one({"Id": item['ProductId']})
        if product:
            predicted_products.append(product)
            predicted_products[-1]['Rating'] = item['rating']

    predicted_products = list(predicted_products)

    return predicted_products

In [175]:
# Test the Function above on predictions excluding the current product on page
user_id = int(189)
product_id = int(92) # Shaun White snowboard
num_results = 10

# Predictions excluding the current product on page
on_page_predictions = predictions_from_current_product_page(user_id, product_id, num_results)
print_predictions_from_product_page(on_page_predictions)



--------Current Page Results: ---------
+------------+--------------------------------+--------+-------------------+
| Product Id |              Name              | Price  |       Rating      |
+------------+--------------------------------+--------+-------------------+
|     42     | Gravity 5000 All-Mountain Skis | 699.0  |  7.15698766708374 |
|     72     | GravityZone All-Mountain Skis  | 699.0  | 7.150111198425293 |
|     22     |     Venture 2022 Snowboard     | 499.0  | 6.249397277832031 |
|     62     |     Shadow Black Snowboard     | 379.0  | 5.958791732788086 |
|     27     |  EcoLodge 45L Travel Backpack  | 129.0  | 5.905089378356934 |
|     88     |   Alpine AlpinePack Backpack   | 129.0  | 5.895949363708496 |
|     32     |    Cosmic Purple Snowboard     | 419.99 | 5.894195079803467 |
|     53     |     Raven Swift Snowboard      | 349.0  | 5.818643569946289 |
|     73     |    Omni-Snow Dual Snowboard    | 289.99 | 5.817547798156738 |
+------------+---------------------

In [192]:
def predictions_from_vector_search(user_id, user_query, num_results=10):
    """ 
    This function takes a user prompt search for products and returns products that are predicted for the user. 
    """
    
    # Generate the embedding for the user query
    query_embedding = generate_embeddings(user_query)

    # Get the predicted products for the user
    predicted_products = predicted_ratings.find_one( { "UserId": user_id } )

    # Convert to a dictionary
    predicted_products = {prediction['ProductId']: prediction for prediction in predicted_products['Predictions']}

    # Filter criteria to include product ids from the predicted products
    filter_criteria = { 
        "Id": {"$in": list(predicted_products.keys())}
    }

    results = product_catalog.aggregate([
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "Embedding",
                    "k": num_results,
                    "filter": filter_criteria
                },
                "returnStoredSource": True
            }},
        {
            '$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' }
        }
    ])

    filtered_vector_search = list(results)

    # Add the Rating field to the documents in filtered_vector_search
    for document in filtered_vector_search:
        product_id = document['document']['Id']
        if product_id in predicted_products:
            document['document']['rating'] = predicted_products[product_id]['rating']

    # Assuming filtered_vector_search is the list you want to extract from
    top_vector_result = filtered_vector_search.pop(0)
    
    # Sort the remaining results by rating
    sorted_vector_search = sorted(
        filtered_vector_search,
        key=lambda document: (-document['document'].get('rating', 0)),
        reverse=False
    )

    # Insert the top result at the beginning of the list
    sorted_vector_search.insert(0, top_vector_result)

    return sorted_vector_search

In [193]:
# Test the function above for filtering vector search results with predicted products

user_query = "Snowboards used by Olympic champions"
user_id = int(189)
num_results = 10

# Vector Search with Predictions
vector_search_with_predictions = predictions_from_vector_search(user_id, user_query, num_results)
print_vector_search_results(vector_search_with_predictions)


---------Vector Search Results: --------
+------------+------------------------------+--------+--------------------+--------------------+
| Product Id |             Name             | Price  |  Similarity Score  |       Rating       |
+------------+------------------------------+--------+--------------------+--------------------+
|     92     |  Shaun White Powder Groomer  | 449.99 | 0.8480039834976196 | 6.024456977844238  |
|     22     |    Venture 2022 Snowboard    | 499.0  | 0.8239901485466837 | 5.9668755531311035 |
|     62     |    Shadow Black Snowboard    | 379.0  | 0.8160758261703205 | 5.747269630432129  |
|     32     |   Cosmic Purple Snowboard    | 419.99 | 0.8276490959702903 | 5.633471965789795  |
|     53     |    Raven Swift Snowboard     | 349.0  | 0.8259191409301261 | 5.577742099761963  |
|     73     |   Omni-Snow Dual Snowboard   | 289.99 | 0.8393915199870792 | 5.572394847869873  |
|     60     | SummitRider Snowboard Boots  | 249.0  | 0.8236519484105107 | 5.37770986